### Imports


In [1]:
import numpy as np
import librosa
import copy
from sklearn.decomposition import PCA
import IPython
import IPython.display as ipd
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io
from heapq import heappush
import heapq
import collections
import pandas as pd

###  Single-channel Source Separation


In [33]:
nmf1,rate1 = librosa.load(root + "trs.wav",sr=None)
nmf2,rate2 = librosa.load(root + "trn.wav",sr=None)
nmf3,rate3 = librosa.load(root + "x_nmf.wav",sr=None)

In [34]:
IPython.display.Audio(root + "trs.wav")

In [35]:
IPython.display.Audio(root + "trn.wav")

In [36]:
IPython.display.Audio(root + "x_nmf.wav")

In [37]:
nmf1_spec = librosa.core.stft(nmf1, n_fft=1024, hop_length=512)
nmf2_spec = librosa.core.stft(nmf2, n_fft=1024, hop_length=512)
X = librosa.core.stft(nmf3, n_fft=1024, hop_length=512)



In [38]:
nmf1_spec = np.abs(nmf1_spec)
nmf2_spec = np.abs(nmf2_spec)
nmf3_spec = np.abs(X)

nmf2_spec.shape

(513, 989)

In [39]:
Ws = np.ones((513,30))
Wn = np.ones((513,30))

In [40]:
Hs = np.ones((30,989))
Hn = np.ones((30,989))


In [41]:
X1 = copy.deepcopy(nmf1_spec)
X2 = copy.deepcopy(nmf2_spec)

lr1 = 2e-7
lr2 = 5e-5

thresh = 0.16
prev_obj = np.inf

In [42]:
prev_obj = np.full((513,989),np.inf)

for i in range(0,5000):

    obj1 = 0.5*np.trace(np.transpose(X1 - np.matmul(Ws,Hs)))
    obj1*= (X1 - np.matmul(Ws,Hs))
    
    Ws -= lr1*(np.matmul(Ws,np.matmul(Hs,np.transpose(Hs))) - np.matmul(X1,np.transpose(Hs)))
    Hs -= lr1*(np.matmul(np.matmul(np.transpose(Ws),Ws),Hs) - np.matmul(np.transpose(Ws),X1))
    if np.allclose(prev_obj,obj1,atol=1e-4,rtol=1e-4):
        print("Converged at ", i)
        break
    prev_obj = copy.deepcopy(obj1)



Converged at  3412


In [43]:
prev_obj = np.full((513,989),np.inf)


for i in range(0,5000):
#     if i%100 == 0:print(i)
    obj2 = 0.5*np.trace(np.transpose(X2 - np.matmul(Wn,Hn)))
    obj2*= (X2 - np.matmul(Wn,Hn))
    
    Wn -= lr2*(np.matmul(Wn,np.matmul(Hn,np.transpose(Hn))) - np.matmul(X2,np.transpose(Hn)))
    Hn -= lr2*(np.matmul(np.matmul(np.transpose(Wn),Wn),Hn) - np.matmul(np.transpose(Wn),X2))
    if i==1000: 
        lr2/=10
    if np.allclose(prev_obj,obj2,atol=1e-3,rtol=1e-3):
        print("Converged at ", i)
        break
    prev_obj = copy.deepcopy(obj2)



Converged at  2967


In [44]:
W = np.column_stack((Ws,Wn))
W.shape

(513, 60)

In [45]:
prev_obj = np.full((513,131),np.inf)

H = np.ones((60,131))
X3 = copy.deepcopy(nmf3_spec)
lr3 = 1e-4
for i in range(0,5000):
    obj3 = 0.5*np.trace(np.transpose(X3 - np.matmul(W,H)))
    obj3*= (X3 - np.matmul(W,H))

    H -= lr3*(np.matmul(np.matmul(np.transpose(W),W),H) - np.matmul(np.transpose(W),X3))

    if i==1500: 
        lr3/=10
    if np.allclose(prev_obj,obj3,atol=1e-3,rtol=1e-3):
        print("Converged at ", i)
        break
    prev_obj = copy.deepcopy(obj3)



Converged at  4537


In [46]:
Y = np.matmul(W,H)
Y.shape

(513, 131)

In [47]:
M_X = X/np.abs(Y)
M_X.shape


(513, 131)

In [48]:
Shat = np.matmul(Ws,H[:30])
Shat.shape


(513, 131)

In [49]:
new_audio = np.abs(Shat)*M_X

Recovered audio from the phase X (question 3.4)

In [50]:
speech_M_X =librosa.istft(new_audio,hop_length=512, length=new_audio.shape[0]*new_audio.shape[1])
librosa.output.write_wav(root + "speech_M_X.wav", speech_M_X, rate3)
IPython.display.Audio(root + "speech_M_X.wav")

In [51]:
Mask = np.matmul(Ws,H[:30])/Y

In [52]:
output = X*Mask
output = output

Recovered audio from magnitude matrix (question 3.5)

In [53]:
speech_output =librosa.istft(output,hop_length=512, length=output.shape[0]*output.shape[1])
librosa.output.write_wav(root + "speech_output.wav", speech_output, rate1)
IPython.display.Audio(root + "speech_output.wav")

Both the methods produced almost similar audios. It is difficult to decide just by hearing them. But, I think the second one will have a better SNR.